In [ ]:
import os
import geopandas as gpd
from shapely.ops import transform
from functools import partial
import pyproj
import pandas as pd

def calculate_distance(geometry):
    if geometry.is_empty:
        return 0
    else:
        # Project the geometry to a CRS with units in meters
        project = partial(
            pyproj.transform,
            pyproj.Proj(init='epsg:4326'),  # WGS84 coordinate reference system
            pyproj.Proj(init='epsg:3857')   # Web Mercator projection with meters as units
        )
        projected_geometry = transform(project, geometry)
        return projected_geometry.length

def create_nodes_along_line(line, interval=20):
    nodes = []
    length = calculate_distance(line)
    
    if length == 0:
        return []

    # Project the line to a CRS with units in meters
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),  # WGS84 coordinate reference system
        pyproj.Proj(init='epsg:3857')   # Web Mercator projection with meters as units
    )
    projected_line = transform(project, line)

    # Generate nodes at 20-meter intervals
    for distance in range(0, int(length), interval):
        point = projected_line.interpolate(distance)
        nodes.append(point)

    return nodes

def create_nodes_dataframe(shapefile_path, interval=20):
    # Read the shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    # Create nodes for each geometry in the shapefile
    all_nodes = []
    for index, row in gdf.iterrows():
        nodes = create_nodes_along_line(row['geometry'], interval)
        all_nodes.extend(nodes)

    # Extract latitude and longitude from each node
    latitudes = [node.y for node in all_nodes]
    longitudes = [node.x for node in all_nodes]

    # Create a DataFrame with latitude and longitude columns
    nodes_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

    return nodes_df

def save_nodes_to_files(nodes_df, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save nodes to CSV file
    csv_path = os.path.join(output_folder, 'output_nodes.csv')
    nodes_df.to_csv(csv_path, index=False)
    print(f'Nodes CSV file saved to {csv_path}')

    # Convert DataFrame to GeoDataFrame
    geometry = gpd.points_from_xy(nodes_df['Longitude'], nodes_df['Latitude'])
    gdf_nodes = gpd.GeoDataFrame(nodes_df, geometry=geometry)

    # Save nodes to Shapefile
    shp_path = os.path.join(output_folder, 'output_nodes.shp')
    gdf_nodes.to_file(shp_path)
    print(f'Nodes Shapefile saved to {shp_path}')

if __name__ == "__main__":
    # Replace 'your_shapefile.shp' with the actual path to your shapefile
    shapefile_path = r"filepath"

    # Replace 'your_folder_name' with the desired folder name
    output_folder = 'NodeOutput20m"

    try:
        nodes_df = create_nodes_dataframe(shapefile_path)
        save_nodes_to_files(nodes_df, output_folder)
    except Exception as e:
        print(f'Error: {e}')
